# Dev_Kits

In [1]:
import http.server
import math
import os
import socketserver
import zipfile
import joblib
import re
import matplotlib.pyplot as plt
import pandas as pd
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from itertools import product
from surprise import Dataset, Reader

# Var

In [2]:
class Config:
    WEB_SERVER_PORT = 8002  # 網頁伺服器的埠號
    WEB_MODE = True  # True: 啟用網頁伺服器; False: 在文字介面直接執行程式



# Sys

In [3]:
class R_sys:
    def __init__(self):
        self.books = pd.read_table("./book_crossing/items_info_new.dat")
        self.user = 1
        self.history = pd.read_table("./book_crossing/book_history_new.dat")
        self.ratings = pd.read_table("./book_crossing/book_ratings_new.dat")
        
    def get_item(self, itemID):
        return self.books[self.books.Book_ID==itemID]

    def get_book_table(self):
        return self.books.copy()

    def set_book_table(self, newTable):
        self.books = newTable
        return

    def get_history_table(self):
        return self.history.copy()

    def set_history_table(self, newTable):
        self.history = newTable
        return
    
    def get_history(self, user, item):
        search = self.history[(self.history.user==user)&(self.history.item==item)]
        return search
    
    def get_rating(self, user, item):
        search = self.ratings[(self.ratings.user==user)&(self.ratings.item==item)]
        if not(search.empty) :
            return search['rating'][0]
        return -1

    def get_rating_table(self):
        return self.ratings.copy()

    def set_rating_table(self, newTable):
        self.ratings = newTable
        return
    
    def set_rating(self, user, item, value):
        self.ratings[(self.ratings.user==user)&(self.ratings.item==item)]['rating']=value
        return


In [4]:
# rec = R_sys()


# Algorithm

In [5]:
# 加載模型
model_filename = './r_sys_svd.pkl'
model = joblib.load(model_filename)

def predict_rating(user_id,item_id):
    #透過矩陣分解預測user_id用戶對item_id書籍的評分
    pre_rating = round(model.predict(user_id, item_id).est, 1)
    return pre_rating

def choose_active_user(target_user):
    #選取前100位最活躍的用戶
    #回傳dataframe
    #讀取資料
    data = rec.get_history_table()

    # 使用value_counts()計算user值的出現次數
    value_counts = data['user'].value_counts(ascending=False)

    # 轉成df
    result_df = pd.DataFrame({'user': value_counts.index, 'count': value_counts.values})

    # 列出前100個活躍用戶
    active_user=result_df.head(50).copy()
    
    if target_user not in active_user['user'].values:
        active_user.loc[len(active_user), 'user'] = target_user
    
    return active_user[['user']]

def ac_user_rating(target_user):
    
    active_user = choose_active_user(target_user)
    #尋找活躍用戶

    item_data = rec.get_book_table()
    item_data = item_data[['Book_ID']].sample(frac=1, random_state=42).head(1000) 
    #索取現有的書本ID

    all_combinations = list(product(active_user['user'], item_data['Book_ID']))
    # 使用 itertools.product 生成所有可能的組合

    result_df = pd.DataFrame(all_combinations, columns=['user', 'item'])
    # 將組合轉換為DataFrame

    rating = rec.get_rating_table().sort_values(by=['user','item'])
    #讀取現有的評分矩陣

    merged_df = pd.merge(result_df, rating, on=['user','item'], how='outer')
    
    merged_df = merged_df[merged_df['user'].isin(active_user['user'])]
    #活躍用戶對於每本書的評分矩陣(無矩陣分解預測內容)
    
    return merged_df

def predict_ac_user_rating(target_user):
    # 將 rating 的 NaN 值填充為預測值
    merged_df = ac_user_rating(target_user).sort_values(by='item')
    merged_df['rating'] = merged_df.apply(lambda row: predict_rating(row['user'], row['item']) if pd.isna(row['rating']) else row['rating'], axis=1)
    return merged_df

def user_similarity(user_ratings):
    # 本函數的目的是計算用戶之間的相似度，基於用戶的協同過濾會用到
    # 計算用戶相似度的方法有很多，這裡使用的是Pearson correlation coefficient
    # 這裡的user_ratings是一個DataFrame，有3個欄位，分別是user, item, rating
    # 這裡的user是用戶的id，item是書籍的id，rating是用戶對書籍的評分
    rating_matrix = user_ratings.pivot(index='user', columns='item', values='rating')
    
    # 計算用戶相似度前，先將每個用戶的評分減去評分的平均值
    rating_matrix = rating_matrix.subtract(rating_matrix.mean(axis=1), axis='rows')
    
    # 轉置矩陣，把每個用戶的評分變成直行，再對每個直行兩兩之間計算 Pearson correlation
    # 也可以用scipy.stats.pearsonr，例如計算編號0和編號3的用戶就用pearsonr(df.loc[0], df.loc[3])
    # Pandas的Pearson算法和Charu課本上的不同，前者取單人平均分數時只看兩個人都評分的書籍，後者則是各用戶自己取平均，不管交集
    user_similarity_pearson = rating_matrix.T.corr(method='pearson')

    return user_similarity_pearson

def top_k_similar_users(user_ratings, target_user, k = 10, similarity_threshold=0.3):
    # 本函數的目的是找出和target_user最相似的k(預設為10)個用戶、閥值預設是0.3，基於用戶的協同過濾會用到

    # 計算用戶相似度
    user_sim = user_similarity(user_ratings)
    rating = rec.get_rating_table()
#     if target_user not in user_sim.index:
#         print('用戶 {:d} 目前沒有評分紀錄，無法找到與其最相似的用戶'.format(target_user))
#         return None
    if target_user not in rating['user'].values:
        print(f'{target_user} 尚未有評分紀錄。')


    # 移除自己這一列，要找的是鄰居，不是自己
    user_sim.drop(index=target_user, inplace=True)

    # 選擇在user_id這一行的相似度分數高於閥值的所有用戶列們(他們和target_user的相似度高於閥值)
    threshold_selector = user_sim.loc[:, target_user] > similarity_threshold
    similar_users = user_sim.loc[threshold_selector, target_user]  # 把他們和target_user的相似度挑出來(搭配他們的index)

    # 將相似度Series的名稱改成similarity
    similar_users.rename('similarity', inplace=True)

    # 排序，相似度高的在前面，只取前k個相似度最大的用戶
    similar_users = similar_users.sort_values(ascending=False).head(10)
#     similar_users = similar_users[:k]

    # 回傳相似度最高的k個用戶，index是這些用戶的user，value是這些用戶和target_user的相似度
    return similar_users

def target_user_unseen(target_user):
    #目標用戶尚未觀看的書籍
    #回傳只有Book_ID的dataframe
    #target_user為目標用戶的ID
    history = rec.get_history_table()
    target_user_accessed = history[history['user'] == target_user][['item']]
    book_item = rec.get_book_table()
    book_item = book_item[['Book_ID']]
    unseen = book_item[~book_item['Book_ID'].isin(target_user_accessed['item'])]
    return unseen

def ac_user_seen(ac_user_series):
    # k個活躍用戶閱覽過的所有書籍
    #回傳只有Book_ID的dataframe
    #ac_user_series為活躍用戶的series，
    #該series默認為method top_k_similar_users()的output
    
    
    if ac_user_series is None:
        return None
    series_index = ac_user_series.index
    history = rec.get_history_table()
    dataframes = []
    
    for i in range(len(series_index)):
        target_user_accessed = history[history['user'] == series_index[i]][['item']]
        dataframes.append(target_user_accessed)
        # 將所有 活躍用戶閱覽過的所有書籍的dataframe 存放在一個列表中

    all_unique_items = set()
    for df in dataframes:
        unique_items_in_df = set(df['item'].unique())
        all_unique_items.update(unique_items_in_df)
        # 用set收集所有 "item" 欄位中的唯一值

    allset = pd.DataFrame(all_unique_items, columns=['Book_ID'])
    return allset.sort_values(by='Book_ID')

def ac_seen_targer_unseen(data, target_user):
    #活躍用戶有閱覽過 但是目標用戶沒有看過的書籍
    #回傳只有Book_ID的dataframe
    
    tu = target_user_unseen(target_user)
    acus = ac_user_seen(top_k_similar_users(data, target_user))
    if acus is None:
        return None
    
    merged_df = pd.merge(acus, tu, on='Book_ID', how='inner')
    return merged_df.sort_values(by = 'Book_ID')

def average_mean_ac_user_rating(target_user):
    #預測評分平均的排序的前10本書
    #target_user為目標用戶的user_id
    #回傳一個list[1o]，內容是10本書的Book_id
    
    user_rating = predict_ac_user_rating(target_user)
    #預測的評分矩陣
    
    new_Book_ID=ac_seen_targer_unseen(user_rating, target_user)
    #取得活躍用戶看過 但是 目標用戶沒看過的書籍
    if new_Book_ID is None:
        return None
    
    take_data=user_rating.copy()
    take_data.rename(columns={'item': 'Book_ID'}, inplace=True)
    #將預測的評分矩陣欄位"item"名稱改為"Book_ID" 方便之後執行
    
    BookRating_info = pd.merge(new_Book_ID, take_data, on='Book_ID', how='inner')
    BookRating_info.drop(columns=['user'], inplace=True)
    #留下 「取得活躍用戶看過 但是 目標用戶沒看過的書籍」 的評分矩陣
    
    avg_ratings = BookRating_info.groupby('Book_ID')['rating'].mean()
    top_10 = avg_ratings.nlargest(10)
    #做平均值 並 選出平均分數最高的前十本書
    
    return top_10.index.tolist()

def pear10(target_user):
    return average_mean_ac_user_rating(target_user)


In [6]:


def target_user_unread(target_user):
    #尋找用戶沒讀過的書籍
    #回傳一個只有item的dataframe
    
    history = rec.get_history_table()
    target_user_history = history.loc[history['user']==target_user][['item']].copy()
    #目標用戶讀過的書籍
    item_data = rec.get_book_table()[['Book_ID']]
    item_data.rename(columns={'Book_ID': 'item'}, inplace=True)
    unread_books = pd.merge(item_data, target_user_history, on='item', how='left', indicator=True).query("_merge == 'left_only'").drop('_merge', axis=1)
    #留下用戶沒讀過的書籍
    
    return unread_books

def matrix_recomment(target_user):
    #利用矩陣分解預測 閱覽數最高的前100本書 並取分數最高的前10本
    #回傳一個list 內容是10本書的Book_ID
    
    history = rec.get_history_table()
    data = target_user_unread(target_user)
    #data為用戶沒讀過的書籍
    merge_df = pd.merge(history, data, on='item', how='inner')
    #得到一個目標用戶沒讀過書籍的紀錄
    
    category_counts = merge_df['item'].value_counts()
    accessed_count = pd.DataFrame(category_counts.index, columns=['item']).head(100)
    #計算每本目標用戶沒讀過書籍的閱覽次數並排序熱門前100本
    
    accessed_count['rating'] = accessed_count.apply(lambda row: predict_rating(target_user, row['item']), axis=1)
    accessed_count.sort_values(by='rating',ascending =False).head(10)
    #為這100本書做矩陣分解預測分數，並取分數最高的前10本書
    top_10 = accessed_count.sort_values(by='rating',ascending =False).head(5)['item'].tolist()
    #轉乘list
    
    return top_10


def popular10(target_user):
    return matrix_recomment(target_user)

# Web

In [7]:
class HTTPServer(socketserver.ThreadingMixIn, http.server.HTTPServer):
    # 這個類別是為了讓HTTPServer可以在執行時，按下Ctrl+C可以正常停止
    def __init__(self, server_address, RequestHandlerClass):
        http.server.HTTPServer.__init__(self, server_address, RequestHandlerClass)
        self.daemon_threads = True
        self.allow_reuse_address = True

In [8]:
class Web:
    # 這個類別是為了啟動Web伺服器，並在伺服器停止時，關閉Python程式，方便在命令列中執行，而不是在IDE中執行
    def __init__(self):
        self.server = None

    def start_server(self):
        # 啟動Web伺服器，並在伺服器停止時，關閉Python程式
        self.server = HTTPServer(('localhost', Config.WEB_SERVER_PORT), RequestHandler)
        print('網頁伺服器已經啟動，請開啟瀏覽器，輸入網址 http://localhost:{}/'.format(Config.WEB_SERVER_PORT))
        print('按下Ctrl+C可以停止伺服器')
        self.server.serve_forever()
        print('網頁伺服器已經停止')

    def stop_server(self):
        # 停止Web伺服器
        print('停止網頁伺服器')
        self.server.shutdown()


In [9]:
class RequestHandler(http.server.BaseHTTPRequestHandler):
    # 用來處理HTTP請求的類別，並覆寫了一些方法，以便處理我們的請求，並回應我們的網頁

    KEY_RATING_PREFIX = 'rating_'  # 用戶對於物品的評分
    KEY_SWITCH_USER = 'switch_user'  # 用戶選擇切換用戶
    IMG_BASE_URL = 'https://image.tmdb.org/t/p/w440_and_h660_face'
    # 用戶對於物品的評分, float32, 0~5, 0.5為單位
    VALID_RATINGS = ( 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

    def show_page_title(self):
        # 顯示網頁標題，以及設定網頁的編碼為 UTF-8，並取消網頁的快取機制，以便即時更新網頁內容，而不是使用快取的內容
        self.wfile.write('<html><head>'.encode('utf-8'))
        self.wfile.write(
            '<meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate" />'.encode('utf-8'))
        self.wfile.write('<title>電影推薦系統實驗平台</title>'.encode('utf-8'))

        # 設定網頁的字型為微軟正黑體，並設定表格的樣式，以便網頁看起來更好看，也更容易閱讀，而不是使用瀏覽器預設的樣式
        self.wfile.write('<style>'.encode('utf-8'))
        self.wfile.write('body { font-family: "微軟正黑體"; }'.encode('utf-8'))
        self.wfile.write('h1 { font-size: 1.5em; }'.encode('utf-8'))
        self.wfile.write('h2 { font-size: 1.2em; }'.encode('utf-8'))
        self.wfile.write('h3 { font-size: 1.0em; }'.encode('utf-8'))
        self.wfile.write('table { border-collapse: collapse; }'.encode('utf-8'))
        self.wfile.write('table, th, td { border: 1px solid black; }'.encode('utf-8'))
        self.wfile.write('th, td { padding: 5px; }'.encode('utf-8'))
        self.wfile.write('th { text-align: left; }'.encode('utf-8'))
        self.wfile.write('</style>'.encode('utf-8'))

        # 設定網頁的圖示為電影的圖示
        self.wfile.write('<link rel=icon href=https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6/svgs/solid/film.svg>'.encode('utf-8'))
        self.wfile.write('</head><body>'.encode('utf-8'))

        # 顯示網頁的標題
        self.wfile.write('<h1>書籍推薦系統</h1>'.encode('utf-8'))
    
    def show_rec_table(self, rec_list):
        self.wfile.write('<table>'.encode('utf-8'))

        self.wfile.write(
                '<tr><th>評分</th><th width=300>標題</th><th>圖片</th><th width=900>簡介</th><th>SVD分數</th></tr>'.encode('utf-8'))
        
        # 逐筆顯示推薦清單的詳細資訊，包括編號、評分、標題、圖片、簡介，以及用戶對這些物品的評分，以便用戶可以修改評分
        for i in rec_list:
            
            item = rec.get_item(i)
            
            self.wfile.write('<tr>'.encode('utf-8'))
            
            #評分
            self.wfile.write('<td><select name="{}{}">'.format(self.KEY_RATING_PREFIX, i).encode('utf-8'))
            self.wfile.write('<option value=""></option>'.encode('utf-8'))
            user_rating = rec.get_rating(rec.user, i)
            for j in self.VALID_RATINGS:
                selected = ' selected' if j == user_rating else ''
                self.wfile.write('<option value="{}"{}>{}</option>'.format(j, selected, j).encode('utf-8'))
            self.wfile.write('</select></td>'.encode('utf-8'))
            
            #標題
            self.wfile.write('<td><a href="/item/{}" >{}</a></td>'.format(i, item['Book-Title'].values[0]).encode('utf-8'))
            #圖片
            self.wfile.write('<td><a href="/item/{}" ><img src="{}"></a></td>'.format(i, item['Image-URL-S'].values[0]).encode('utf-8'))
            #簡介
            self.wfile.write('<td>{}</td>'.format(item['Summary'].values[0]).encode('utf-8'))
            #SVD分數
            self.wfile.write('<td>{}</td>'.format(predict_rating(rec.user, i)).encode('utf-8'))
            
            self.wfile.write('</tr>'.encode('utf-8'))
        
        self.wfile.write('</table>'.encode('utf-8'))

        
    def show_page_footer(self):
        # 顯示頁尾
        self.wfile.write('<hr>'.encode('utf-8'))
        self.wfile.write('<p>本系統改進自 <a href="https://linjiun.github.io/" target="blank">Linjiun Tsai</a> 製作之電影推薦系統實驗平台，'
                         '所有書籍相關資料取自BookCrossing公開資料集，版權歸原作者。</p>'.encode('utf-8'))
        self.wfile.write('</body></html>'.encode('utf-8'))
        
    
    def show_main_page(self):
        # 顯示推薦系統主頁面，包括標題、用戶最喜歡的電影、推薦清單，以及用戶可以給予評分的表單

        # 設定HTTP回應的狀態碼、標頭和內容類型
        self.send_response(200)
        self.send_header('Content-type', 'text/html; charset=utf-8')
        self.end_headers()

        # 顯示標題
        self.show_page_title()
        
        #表單開始
        self.wfile.write('<form action="/feedback" method="post">'.encode('utf-8'))
        
        #顯示推薦表格
        self.wfile.write('<h2>熱門書籍</h2>'.encode('utf-8'))
        self.show_rec_table(popular10(rec.user))
        
        self.wfile.write('<h2>用戶 {} 最喜歡的書籍</h2>'.format(rec.user).encode('utf-8'))
        self.show_rec_table(pear10(rec.user))
        
        # 顯示提交表單的按鈕
        self.wfile.write('<input type="submit" value="更新評分" />'.encode('utf-8'))
        self.wfile.write('<input type="hidden" name="user_id" value="{}" />'.format(rec.user).encode('utf-8'))
        self.wfile.write('</form>'.encode('utf-8'))

        # 顯示更新推薦清單的按鈕
        self.wfile.write('<p><a href="refresh"><input type="button" value="更新推薦清單" /></a></p>'.encode('utf-8'))
        
        # 顯示頁尾
        self.show_page_footer()
    
    def show_item_page(self, item_id):
        # 設定HTTP回應的狀態碼、標頭和內容類型
        self.send_response(200)
        self.send_header('Content-type', 'text/html; charset=utf-8')
        self.end_headers()
        
        item = rec.get_item(item_id)
        
        # 顯示標題
        self.show_page_title()
        
        # 大圖
        self.wfile.write('<p><img src="{}"></p>'.format(item['Image-URL-L'].values[0]).encode('utf-8'))
        
        # 標題
        self.wfile.write('<p><h2>{}</h2></p>'.format(item['Book-Title'].values[0]).encode('utf-8'))
        
        # 說明
        self.wfile.write('<p>{}</p>'.format(item['Summary'].values[0]).encode('utf-8'))
        
        #評分
        self.wfile.write('<form action="/feedback" method="post">'.encode('utf-8'))
        self.wfile.write('<p><select name="{}{}">'.format(self.KEY_RATING_PREFIX, item_id).encode('utf-8'))
        self.wfile.write('<option value=""></option>'.encode('utf-8'))
        user_rating = rec.get_rating(rec.user, item_id)
        for j in self.VALID_RATINGS:
            selected = ' selected' if j == user_rating else ''
            self.wfile.write('<option value="{}"{}>{}</option>'.format(j, selected, j).encode('utf-8'))
        self.wfile.write('</select></p>'.encode('utf-8'))
        
        # 顯示提交表單的按鈕
        self.wfile.write('<input type="submit" value="更新評分" />'.encode('utf-8'))
        self.wfile.write('<input type="hidden" name="user_id" value="{}" />'.format(rec.user).encode('utf-8'))
        self.wfile.write('</form>'.encode('utf-8'))
        
        #首頁按鈕
        self.wfile.write('<p><a href="/"><input type="button" value="回到首頁" /></a></p>'.encode('utf-8'))
        
        # 顯示頁尾
        self.show_page_footer()
        
    def do_GET(self):
        print('收到HTTP GET請求，路徑是', self.path)
        is_get_item = re.match(r'/item/(\d+)', self.path)
        
        # 依據路徑的不同，顯示不同的頁面

        # 如果路徑是根目錄，則顯示主頁面
        if self.path == '/':
            self.show_main_page()
        
        elif self.path == '/refresh':

            # 設定HTTP回應的狀態碼、標頭和內容類型
            self.show_main_page()

            # 顯示更新推薦清單的訊息
            self.wfile.write('<p>已經更新推薦清單</p>'.encode('utf-8'))
            self.wfile.write('<p><a href="/"><input type="button" value="回到首頁" /></a></p>'.encode('utf-8'))

            # 顯示頁尾
            self.show_page_footer()
        
        elif self.path == '/quit':

            # 設定HTTP回應的狀態碼、標頭和內容類型
            self.send_response(200)
            self.send_header('Content-type', 'text/html; charset=utf-8')
            self.end_headers()
            self.show_page_title()

            # 顯示結束系統的訊息
            self.wfile.write('<p>系統已經終止，請確認Python程式已經完全關閉，以免無法重啟Web伺服器。</p>'.encode('utf-8'))
            self.wfile.write('<p><a href="/"><input type="button" value="回到首頁" /></a></p>'.encode('utf-8'))

            # 顯示頁尾
            self.show_page_footer()

            # 停止伺服器
            web.stop_server()
        
        elif is_get_item:
            self.show_item_page(int(is_get_item.group(1)))
            rec.set_history_table(rec.history._append({'user':rec.user, 'item':int(is_get_item.group(1)), 'accessed':1 }, ignore_index=True))
            
        # 其他路徑則顯示404錯誤
        else:
            self.send_response(404)
            self.send_header('Content-type', 'text/html; charset=utf-8')
            self.end_headers()
            self.show_page_title()
            self.wfile.write('<p>你走錯地方了(404)</p>'.encode('utf-8'))
            self.wfile.write('<p><a href="/"><input type="button" value="回到首頁" /></a></p>'.encode('utf-8'))
            self.wfile.write('</body></html>'.encode('utf-8'))

    def do_POST(self):
        print('收到HTTP POST請求，路徑是', self.path)
        # 依據路徑的不同，處理不同的表單，並顯示不同的頁面

        # 如果路徑是/feedback，則處理用戶的評分表單
        if self.path == '/feedback':
            # 讀取表單內容
            content_len = int(self.headers.get('Content-Length'))
            post_body = self.rfile.read(content_len)
            post_body = post_body.decode('utf-8')
            print('收到的表單內容是', post_body)

            # 解析表單內容，例如：rating_1=5&rating_2=4&rating_3=3，並更新用戶的評分紀錄
            for item in post_body.split('&'):
                key, value = item.split('=')

                # 如果評分是空字串(用戶沒有給分)，則跳過
                if value == '':
                    continue

                # 如果是有數值的評分欄位，則更新用戶的評分紀錄
                if key.startswith(self.KEY_RATING_PREFIX):
                    # 例如：rating_1=4.5，則取出1，並將評分4.5設定給movie_id=1的電影
                    book_id = int(key[len(self.KEY_RATING_PREFIX):])
                    if rec.get_rating(rec.user, book_id)==-1:
                        rec.set_rating_table(rec.ratings._append({'user':rec.user, 'item':book_id, 'rating':int(value) }, ignore_index= True))
                    else:
                        rec.set_rating(rec.user, book_id, int(value))
                    if rec.get_history(rec.user, book_id).empty:
                        rec.set_history_table(rec.history._append({'user':rec.user, 'item':book_id, 'accessed':1 }, ignore_index=True))
           

            # 設定HTTP回應的狀態碼、標頭和內容類型
            self.send_response(200)
            self.send_header('Content-type', 'text/html; charset=utf-8')
            self.end_headers()
            self.show_page_title()

            # 顯示更新用戶的評分紀錄的訊息
            self.wfile.write('<p>已經更新用戶的評分紀錄，可以繼續更新其他評分。若有需要，可自行啟動推薦清單的更新。</p>'.encode('utf-8'))
            self.wfile.write('<p><a href="/"><input type="button" value="回到首頁" /></a></p>'.encode('utf-8'))

            # 顯示頁尾
            self.show_page_footer()





# Main

In [10]:
if __name__ == '__main__':
    rec = R_sys()
    rec.user = int(input('Login Account:'))
    web = Web()
    web.start_server()


Login Account:5
網頁伺服器已經啟動，請開啟瀏覽器，輸入網址 http://localhost:8002/
按下Ctrl+C可以停止伺服器
收到HTTP GET請求，路徑是 /


127.0.0.1 - - [08/Jan/2024 06:43:50] "GET / HTTP/1.1" 200 -


收到HTTP GET請求，路徑是 /item/4


127.0.0.1 - - [08/Jan/2024 06:44:01] "GET /item/4 HTTP/1.1" 200 -


收到HTTP GET請求，路徑是 /quit
停止網頁伺服器


127.0.0.1 - - [08/Jan/2024 06:44:22] "GET /quit HTTP/1.1" 200 -


網頁伺服器已經停止
